In [1]:
from collections import defaultdict
import numpy as np
from cyvcf2 import VCF
import pandas as pd
from functools import reduce

In [2]:
### Loading coding TR coordinates ###
coordinates = pd.read_csv("TR_intersect.txt", sep = "\t", header=None)

In [3]:
one_nr_per_sample_coding = defaultdict(int)
both_nr_per_sample_coding = defaultdict(int)

for chrom in range(1,23):
    vcf_addr = f"/expanse/projects/gymreklab/helia/ensembl/experiments/coding_regions/heterozygosity/coding_calls_{chrom}.vcf.gz"
    vcf = VCF(vcf_addr)
    samples = vcf.samples
    for record in vcf:
        if int(record.POS) in list(coordinates[1]) and int(record.INFO['END']) in list(coordinates[2]):
            genotypes = record.genotypes
            for i in range(len(samples)):
                sample = samples[i]
                genotype = genotypes[i]
                if genotype[0] == -1:
                    continue
                if genotype[0] != 0 and genotype[1] != 0:
                    both_nr_per_sample_coding[sample] += 1
                    continue
                if genotype[0] != 0 or genotype[1] != 0:
                    one_nr_per_sample_coding[sample] += 1

                

In [4]:
one_nr_per_sample_whole = defaultdict(int)
both_nr_per_sample_whole = defaultdict(int)


for i in range(1,23):
    gt_file = f"/expanse/projects/gymreklab/helia/ensembl/experiments/stats/info/gt_chr{i}.txt"
    with open(gt_file) as f:
        for line in f:
            line = line.split()
            GT = line[3].split("/")
            if GT[0] == ".":
                continue
            if GT[0] != '0' and GT[1] != '0':
                both_nr_per_sample_whole[line[2]] += 1
                continue
            if GT[0] != '0' or GT[1] != '0':
                one_nr_per_sample_whole[line[2]] += 1

In [5]:
### Merging all data ###
one_nr_per_sample_coding_df = pd.DataFrame(one_nr_per_sample_coding.items())
both_nr_per_sample_coding_df = pd.DataFrame(both_nr_per_sample_coding.items())

one_nr_per_sample_whole_df = pd.DataFrame(one_nr_per_sample_whole.items())
both_nr_per_sample_whole_df = pd.DataFrame(both_nr_per_sample_whole.items())

all_df_list = [one_nr_per_sample_coding_df, both_nr_per_sample_coding_df, 
               one_nr_per_sample_whole_df, both_nr_per_sample_whole_df]

all_df = reduce(lambda  left,right: pd.merge(left,right,on=[0],
                                            how='inner'), all_df_list)

all_df.columns = ['sample', 'coding_one_allele_non_ref', 'coding_two_alleles_non_ref',
                  'wg_one_allele_non_ref', 'wg_two_alleles_non_ref']



In [11]:
#### Loading pedigree ####

pedigree = pd.read_csv("/expanse/projects/gymreklab/helia/TR_1000G/1000G.ped", delim_whitespace=True)
pedigree = pedigree[['SampleID', 'Population', 'Superpopulation']]

H3Africa_names = pd.read_csv("/expanse/projects/gymreklab/helia/H3Africa/names/H3A_Baylor_sample_country.txt", header=None, delim_whitespace=True)
H3Africa_names['Superpopulation'] = "H3Africa"
H3Africa_names.columns = ['SampleID', 'Population', 'Superpopulation']

all_samples = pd.concat([pedigree, H3Africa_names])
all_samples
    
all_df_family = pd.merge(all_df, all_samples, left_on = "sample",
                         right_on = "SampleID")[['sample', 'Population', 'Superpopulation',
                                                 'coding_one_allele_non_ref', 'coding_two_alleles_non_ref',
                                                 'wg_one_allele_non_ref', 'wg_two_alleles_non_ref']]
all_df_family
all_df_family.to_csv("tables/Supplementary_Table_6.csv", index = False, sep = "\t")

In [9]:
print(np.mean(all_df_family['wg_one_allele_non_ref']), np.mean(all_df_family['wg_two_alleles_non_ref']))

183899.8861971831 184643.7090140845


In [10]:
print(np.mean(all_df_family['coding_one_allele_non_ref']), np.mean(all_df_family['coding_two_alleles_non_ref']))

158.94732394366198 128.78704225352112
